In [1]:
#ipynb 파일이 분리되어 있는 관계로 이전 파일의 실행 내역 복사

import pandas as pd
data = pd.read_csv("turnover.csv")

data.salary = data.salary.astype('category')
data.salary = data.salary.cat.reorder_categories(['low', 'medium', 'high'])
data.salary = data.salary.cat.codes

departments = pd.get_dummies(data.department)
departments = departments.drop("technical", axis=1)
data = data.drop("department", axis=1)
data = data.join(departments)

target = data.left
features = data.drop("left", axis=1)
from sklearn.model_selection import train_test_split
target_train, target_test, features_train, features_test = train_test_split(target, features, test_size=0.25)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

model = DecisionTreeClassifier(random_state=42)
model.fit(features_train, target_train)

model_best = DecisionTreeClassifier(max_depth=7, class_weight='balanced', random_state=42)
model_best.fit(features_train,target_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

## 최선의 퇴직율 예측 모델 선택

### Hyperparameter tuning

Hyperparameter : 데이터를 학습시킬 때 사람이 직접 튜닝을 해 줘야 하는 변수들</p>
예) max_depth를 어떻게 잡아야 최적인지 알 수가 있나?

<b>Hyperparameter를 튜닝하는 방법</b>
1. Grid Search :
Hyperparameter의 대략적인 법위를 지정하고, 일정한 간격으로 값을 선택하여 학습
2. Random Search : 
Hyperparameter의 대략적인 범위를 지정하고 그 범위내에서 무작위로 값을 선택하여 학습
3. Bayesian optimization : 
기존 학습결과로 Hyperparameter의 산전분포를 가정하고, 최적의 Hyperparameter로 가정되는 값의 학습결과를 얻은 후에 다시 사후분포를 결정하는 작업을 반복

### Cross-validation : sklearn을 사용

GridSearch를 그대로 사용하면 overfitting의 위험성이 있음</p>
=> 테스트 데이터셋을 여러 개(=k개)로 분리하여 각각 번갈아가며 훈련데이터셋과 테스트데이터셋으로 사용

In [2]:
# cross validation을 수행할 함수 호출 : cross_val_score
from sklearn.model_selection import cross_val_score

# 데이터셋 갯수(k)를 10개(fold)로 했을 때 cross validation 점수를 구한다
print(cross_val_score(model, features, target, cv=10))

[ 0.98467688  0.988       0.97133333  0.96333333  0.956       0.982       0.988
  0.99133333  1.          1.        ]


### <b>최적의 hyperparameter를 찾기 위한 여정</b> : GridSearchCV</p>
1. maximum depth와 minimum sample size 등의 hyperparameter에 대입 가능한 수의 값을 만들어서 통제하고</br>
2. 'parameters'라는 딕셔너리를 생성</p>

#### 파라미터 딕셔너리 생성

In [3]:
# max_depth를 위한 값 생성
depth = [i for i in range(5,21)]

# minimum sample size를 위한 값 생성
samples = [i for i in range(50,500,50)]

# 위의 두 값을 파라미터로 사용할 딕셔너리 생성
parameters = dict(max_depth = depth, min_samples_leaf = samples)

#### <b>GridSearch를 통해 최적의 파라미터 도출</b>

In [4]:
# GridSearch를 구현할 함수 호출
from sklearn.model_selection import GridSearchCV

# 파라미터 설정 - 앞에서 설정
# parameters = dict(max_depth = depth, min_samples_leaf = samples)

# 앞에서 작성한 기본 모델과 파라미터를 사용해서 'param_search' 함수 초기화
param_search = GridSearchCV(model, parameters)

# param_search를 훈련 데이터셋에 fitting
param_search.fit(features_train, target_train)

# 최적의 parameter 출력
print(param_search.best_params_)

{'max_depth': 6, 'min_samples_leaf': 50}


### predicting attrition에 필요한 중요한 특성

<b>Feature importances</b></p>
Decision Tree가 만들어지면 sklearn은 feature importance를 쉽게 계산할 수 있음
- importance = 선택된 feature에 대한 Gini의 상대적인 감소
- 모든 feature에 대한 계산이 끝나면 전체 값은 다시 100%로 설정됨
- %간 높은 feature일수록 더 중요 -> 중요하지 않은 feature는 제거하는 게 나음

<b>※ HR Analytics에서 Decision Tree가 많이 쓰이는 이유</b></p>
=> interpretability(해석편리성)
즉, 예측(여기서는 퇴직) 관련 개별 feature의 영향력을 수량화하여 그 중에서 어떤 게 중요한 건지를 식별할 수 있게 함


### 중요도 해석
#### 중요한 feature의 정렬

In [5]:
# feature importances 계산
feature_importances = model_best.feature_importances_

# feature 목록 생성
feature_list = list(features)

# feature list를 DataFrame안에 집어넣기
relative_importances = pd.DataFrame(index=feature_list, data = feature_importances, columns = ["importance"])

# 가장 중요한 featuer를 먼저 보기 위해 DataFrame 정렬
relative_importances.sort_values(by="importance", ascending=False)

,importance
satisfaction_level,0.459330
time_spend_company,0.368838
last_evaluation,0.102196
average_montly_hours,0.042739
number_project,0.023321
sales,0.001745
salary,0.001575
support,0.000257
promotion_last_5years,0.000000
RandD,0.000000


#### 중요한 feature를 선택하여 적용

In [6]:
# relative_importance가 1% 이상인 feature만 선택
selected_features = relative_importances[relative_importances.importance > 0.01]

# 위의 선택된 feature들을 list로 만들기
selected_list = selected_features.index

# 위의 선택된 feature(selected_list)를 features_train 데이터와 features_test 데이터에 적용
features_train_selected = features_train[selected_list]
features_test_selected = features_test[selected_list]

In [7]:
print(features_train_selected)

       satisfaction_level  last_evaluation  number_project  \
2032                 0.54             0.94               4   
1160                 0.36             0.73               4   
11615                0.87             0.89               4   
2166                 0.75             0.70               3   
5668                 0.77             0.55               3   
4564                 0.52             0.47               3   
10250                0.64             1.00               4   
854                  0.79             0.90               5   
1941                 0.81             0.85               4   
13349                0.75             0.93               3   
12091                0.89             1.00               5   
5520                 0.85             0.57               4   
13550                0.76             0.81               4   
7536                 0.51             0.93               3   
547                  0.37             0.50               2   
9038    

### 최적의 모델 개발하고 테스트

1. 앞의 model 평가에서 최적화된 파라미터로 선정된 것은 아래와 같다.
 - max_depth = 5,
 - min_samples = 150,
 - class_weight = "balanced"
2. 그리고 위에서 어떤 feature가 더 중요하고 덜 중요한지도 살펴봤다.
 - selected_list : satisfaction_level, last_evaluation, number_project, average_montly_hours, time_spend_company

In [8]:
# 위의 1에서 확인한 파라미터로 best_model 초기화
model_best = DecisionTreeClassifier(max_depth=6, min_samples_leaf=150, class_weight="balanced", random_state=42)

# 선택된 feature들로 훈련 데이터를 대상으로 best model을 fitting
model_best.fit(features_train_selected, target_train)

# 선택된 feature들로 테스트 데이터를 대상으로 예측 수행
prediction_best = model_best.predict(features_test_selected)


# model의 정확도(accuracy) 점수, recall 점수, ROC/AUC 점수 출력
print(model_best.score(features_test_selected, target_test)*100)
print(recall_score(prediction_best, target_test)*100)
print(roc_auc_score(prediction_best, target_test)*100)

95.4133333333
88.7810140237
93.1861145216
